In [25]:
import SSLimPy.interface.sslimpy as sslimpy
import numpy as np
import astropy.units as u
from copy import copy
import matplotlib.pyplot as plt

# Compute Cosmology at the fiducial value

In [15]:
cosmodict={"h":0.6774,
           "Omegam":0.31,
           "Omegab":0.05,
           "As":2.142e-9,
           "mnu":0.06,
           "ns":0.967}

settings = {"code":"class",
            "do_RSD" : True,
            "nonlinearRSD" : True,
            "FoG_damp" : "ISTF_like",
            "halo_model_PS" : True,
            "Smooth_window" : False,
            "output" : ["Power spectrum"],
            "TracerPowerSpectrum":"clustering",
            }

In [16]:
SSLIMPY = sslimpy.sslimpy(settings_dict=settings,
                           cosmopars=cosmodict,
                           specifications="input/survey_files/forecast_test.yaml")

#--------------------------------------------------#

  SSSSS   SSSSS  L       i            PPPP   y    y 
 S     S S     S L            m   m   P    P y   y  
 S       S       L     iii   m m m m  P    P  y y   
  SSSSS   SSSSS  L       i   m  m  m  PPPP     y    
       S       S L       i   m     m  P        y    
 S     S S     S L       i   m     m  P       y     
  SSSSS   SSSSS  LLLLL iiiii m     m  P      y      

#--------------------------------------------------#


Just to show how to do a fisher forecast we choose following free parameters

$\left\lbrace\alpha_\parallel, \alpha_\perp,  \sigma_v, T_{\rm CO}, b_{\rm CO}, A_s, \right\rbrace$

In [17]:
fidBAO_dict = {
    "alpha_par": 1,
    "alpha_perp": 1,
    "sigma_v": 5 * u.Mpc**2,
    "bmean": 2,
    "Tmean": 0.4 * u.mK,
    }

Obtain fiducial power spectrum and covariance

In [18]:
Fiducial_output = SSLIMPY.compute(cosmopars=cosmodict,
                                  astropars=dict(),
                                  BAOpars=fidBAO_dict,
                                  output=["Power spectrum",
                                          "Covariance"])


Done!


/home/sefa/Schreibtisch/LIM-code/SSLimPy/SSLimPy/LIMsurvey/PowerSpectra.py:749: RuntimeWarning: divide by zero encountered in log
  logP = np.log(Pk_Obs.value)
/home/sefa/Schreibtisch/LIM-code/SSLimPy/SSLimPy/interface/sslimpy.py:89: UserWarning: Output Power spectrum asked for not recognised 
 Skiped
  warn("Output {} asked for not recognised \n Skiped".format(obs))


In [27]:

Ps_fid = Fiducial_output["Power spectrum"]
PfidMoments = np.array([Ps_fid.Pk_0bs,Ps_fid.Pk_2bs,Ps_fid.Pk_4bs])
Cov = Fiducial_output["Covariance"].gaussian_cov()

In [20]:
derivative_dic = dict()
eps = 1e-2

# Cosmo Derivs

In [21]:
cosmo_free_pars = ["As"]

for param in cosmo_free_pars:
    cosmo_var_dict = copy(cosmodict)
    
    # do step
    cosmo_var_dict[param] = cosmodict[param] * ( 1 + eps)
    # compute power spectrum
    Pobs = SSLIMPY.compute(cosmopars=cosmo_var_dict,
                                    astropars=dict(),
                                    BAOpars=fidBAO_dict,
                                    )["Power spectrum"]
    dPp = np.array([Pobs.Pk_0bs, Pobs.Pk_2bs, Pobs.Pk_4bs])

    cosmo_var_dict[param] = cosmodict[param] * ( 1 - eps)
    Pobs = SSLIMPY.compute(cosmopars=cosmo_var_dict,
                                    astropars=dict(),
                                    BAOpars=fidBAO_dict,
                                    )["Power spectrum"]
    dPm = np.array([Pobs.Pk_0bs, Pobs.Pk_2bs, Pobs.Pk_4bs])

    derivative_dic[param] = (dPp - dPm) / (2 * eps * cosmodict[param])

Done!
Done!


# Nuisance Derivs

In [22]:
nuiscance_free_pars = [ "alpha_par", "alpha_perp", "sigma_v", "bmean", "Tmean"]

for param in nuiscance_free_pars:
    BAO_var_dict = copy(fidBAO_dict)
    
    # do step
    BAO_var_dict[param] = BAO_var_dict[param] * (1 + eps)
    # compute power spectrum
    Pobs = SSLIMPY.compute(cosmopars=cosmodict,
                           astropars=dict(),
                           BAOpars=BAO_var_dict,
                           )["Power spectrum"]
    dPp = np.array([Pobs.Pk_0bs, Pobs.Pk_2bs, Pobs.Pk_4bs])

    BAO_var_dict[param] = BAO_var_dict[param] * (1 - eps)
    Pobs = SSLIMPY.compute(cosmopars=cosmodict,
                           astropars=dict(),
                           BAOpars=BAO_var_dict,
                           )["Power spectrum"]
    dPm = np.array([Pobs.Pk_0bs, Pobs.Pk_2bs, Pobs.Pk_4bs])

    derivative_dic[param] = (dPp - dPm) / (2 * eps * fidBAO_dict[param])

Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
